In [ ]:
import numpy as np
from scipy.optimize import linprog

def CVaR_optimization(expected_returns, return_std_devs, alpha, num_scenarios, 
                      benchmark_weights, max_deviation, 
                      sector_weights, max_sector_weight, 
                      credit_weights, max_credit_weight,
                      liquidity_weights, liquidity_constant, x0, max_change):
    
    #x0 is previous weights

    num_bonds = 100  # Number of bonds in the portfolio
    
    # Simulate returns using a normal distribution
    simulated_returns = np.random.normal(loc=expected_returns, scale=return_std_devs, size=(num_scenarios, num_bonds))
    
    # Objective function coefficients: [zeros for bond weights, 1/(1-alpha) for scenarios, 1 for auxiliary variable t]
    c = np.concatenate([np.zeros(num_bonds), np.ones(num_scenarios) / ((1 - alpha) * num_scenarios), [1]])
    
    # Inequality constraints for CVaR: -simulated_returns * weights <= -VaR (scenario losses + t)
    A_ub = np.block([[-simulated_returns, -np.eye(num_scenarios), -np.ones((num_scenarios, 1))]])
    b_ub = np.zeros(num_scenarios)

    # Equality constraint: Sum of portfolio weights must be 1 (full investment)
    A_eq = np.concatenate([np.ones(num_bonds), np.zeros(num_scenarios + 1)]).reshape(1, -1)
    b_eq = [1]

    # Bounds for bond weights and auxiliary variables (non-negative weights and scenario losses)
    bounds = [(0, None)] * num_bonds + [(0, None)] * num_scenarios + [(None, None)]

    # Benchmark deviation constraint
    def deviation_from_benchmark_constraint(weights):
        return max_deviation - np.sum(np.abs(weights[:num_bonds] - benchmark_weights))

    # Sector weight constraint
    def sector_weight_constraint(weights):
        return max_sector_weight - np.sum(weights[:num_bonds] * sector_weights)

    # Credit rating constraint
    def credit_rating_constraint(weights):
        return max_credit_weight - np.sum(weights[:num_bonds] * credit_weights)

    # Liquidity constraint
    def liquidity_constraint(weights):
        return liquidity_constant - np.sum(weights[:num_bonds] * liquidity_weights)
    
    def turnover_constraint(weights, previous_weights, max_change):
        return max_change - np.sum(np.abs(weights[:num_bonds] - previous_weights))

    # Define additional linear constraints based on sector, credit, and liquidity constraints
    additional_constraints = [
        {"type": "ineq", "fun": deviation_from_benchmark_constraint},
        {"type": "ineq", "fun": sector_weight_constraint},
        {"type": "ineq", "fun": credit_rating_constraint},
        {"type": "ineq", "fun": liquidity_constraint},
        {"type": "ineq", "fun": lambda w: turnover_constraint(w, x0, max_change)}  # Turnover constraint
    ]

    # Optimization using linprog (including the additional constraints)
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs', options={'disp': True}, constraints=additional_constraints)
    
    # Extract the optimal portfolio weights (if successful)
    if result.success:
        optimal_weights = result.x[:num_bonds]
        return optimal_weights
    else:
        raise ValueError("Optimization failed: " + result.message)

# Example constraint data
#benchmark_weights = np.random.random(100)  # CHANGE TO REAL
#benchmark_weights /= np.sum(benchmark_weights)  # Normalize to sum to 1
#max_deviation = 0.10  # Maximum allowed deviation from benchmark

#sector_weights = np.random.random(100)  # CHANGE TO REAL
#max_sector_weight = 0.20  # Max sector allocation limit

#credit_weights = np.random.random(100)  # CHANGE TO REAL
#max_credit_weight = 0.25  # Max credit exposure

#liquidity_weights = np.random.random(100)  # Example liquidity weights
#liquidity_constant = 0.10  # Max liquidity constraint

